**Types of Data Anomalies:** 

\- Update anomaly; when you have redundant data and only partially update the data (only updated addresses on some of the author's books)

\- Insertion Anomaly: can't insert data because it's missing other data in a row (Didn't have the author's address when adding a new book, can't add the book)

\- Deletion Anomaly: unintentionally lose data because we've deleted other data (We deleted the books, now the author has disappeared, even though we want to keep them on file)

**Avoiding Anomalies: Normalization Rules**

First Normal Form - Values in each column of a table must be atomic

Second Normal Form - All attributes not part of the key depend on the key

Third Normal Form - No transitive dependencies

\----

Normalized tables in a schema should show relations to the table 1:1, 1:N, ... 

**OLTP - Online Transaction Processing Systems**

\- many reads/writes

\- many processes

\- often normalized to third normal form 

**Analytical DBs**

\- data analysis

\- many reads by many processes

\- many writes by few processes 

\- Often Denormalized

**Denormalization** 

Can significantly improve performance

Why?

\- Data is often redundant

\- Contains non-atomic values

\- Tolerate transitive dependencies

\- Also has reduced risk of anomalies due to:

> \- few updates
> 
> \- batch inserts; data transformed before the inserts
> 
> \- streaming inserts; simple data structures
> 
> \- eliminate the need for complex joins

**Forms of Denormalized DBs**

\- Star Schema: Fact and Dimension tables

> \- Row-level orientation
> 
> \- Columnar data stores

\- Wide Column: everything in a table that is really wide

> \- The opposite of Tidy Data

**Partitioning**

**Vertical Partitioning**

\- Separating by column

\- Increases the number of rows in a data block with fewer columns

\- Global indexes for each partition

\- Can reduce I/O

  

**Horizontal Partitioning**  

\- Separation by subsets of rows \*most common

\- Limits scans to a subset of partitions by chunks of rows, same columns

\- Local indexes for each partition (smaller indexes)

\- Efficient adding and deleting 

  

**Range Partitioning -** type of horizontal

\- Parition on non-overlapping keys (like by month)

\- Parition by date is common

\- Numeric ranges are often used

\- Alphabetic, global region... 

\- _Partition on a value or list of values_

  

**Hash Partitioning**

\- Partition on modulus of hash of parition key

\- Pick the key based on the remainer of the modulus

\- Does not logically group into subgroups

\- _Want an even distribution of data across partitions_

In [1]:
-- Postgres, create a table with partitions and then their partitions

CREATE TABLE iot.sensor_msmt (
    sensor_id int NOT NULL, 
    msmt_date date NOT NULL, 
    temperature int, 
    humidity int)
    PARTITION BY RANGE (msmt_date);

-- It is partitioning by date, now we have to create the partitions
CREATE TABLE io_sensor_msmt_y2021m01 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021_01_01') TO ('2021_01_31');

CREATE TABLE io_sensor_msmt_y2021m02 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021_02_01') TO ('2021_02_28');

: schema "iot" does not exist

Total execution time: 00:00:00.005

**Materialized Views**

Persisited Results of a Query - a form a caching

  

\- Execute the query once

\- Save results once

\- Read many times

\- Trading space for time

  

**When to use materialized views?** 

\- Long-running queries

\- Complex queries, many joins

\- Computing aggregates or other derived data

\- Separate read and write operations

  

**When NOT to use Materialized Views?**

\- Eventual Consistency: you may not have the latest data in an updating system

\- Cost of update process

\- Concurrent reads during update? (Default in postgres)

\- Size of materialized view

\- Refresh Frequency

In [ ]:
-- Create a materialized view - kind of like a WITH clause

CREATE MATERIALIZED VIEW landon.mv_locations_expenses AS 
(
SELECT  l.hotel, 
        l.city, 
        l.state_province, 
        l.country, 
        e.year, 
        e.annual_payroll, 
        e.health_insurance, 
        e.supplies
FROM    
    landon.locations l
    LEFT JOIN
    landon.expenses e 
    ON (l.hotel_id = e.hotel_id)
)

SELECT * FROM landon.mv_locations_expenses;

-- IF there is an update, we can refresh the query

REFRESH MATERIALIZED VIEW landon.mv_locations_expenses;

**Read Replicas**

The primary server is responsible for all read and write queries at the same time

A **read replica** - data written to the primary is written to another instance of the data server

  

Using this, only WRITEs go to the primary server

Primary WRITEs to the Read Replica

All READ queries access the Read Replica  

\- Primary can focus on the writes

\- Multiple replicas can scale to meet read load

\- Especially useful when more read than write workload

\- Need to consider eventual consistency

**CHALLENGE**

An IoT company collects streaming data from thousands of sensors every minute

Data scientists will perform series analyses, including many aggregate data by sensor over hours and days

\- Low Latency is essential

\- Access to all data older than one hour

  

Design 

\- High-level model

\- What kind of structures

\- What kind of design patterns

\-------------------------------------------------------------------

  

\- Sensor data written to table with the attributes for id and measures

\- Partition by time

\- Use Read Replicas to not bog down the server

\- Use Materialized Views, refresh once per hour for hourly